In [2]:
from game import State
from pv_mcts import pv_mcts_scores
from dual_network import DN_OUTPUT_SIZE
from datetime import datetime
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from pathlib import Path
import numpy as np
import pickle
import os

ModuleNotFoundError: ignored

In [ ]:
SP_GAME_COUNT = 1
SP_TEMPERATURE = 1.0

In [ ]:
#先手プレイヤーの価値
def first_player_value(ended_state):
  if ended_state.is_lose():
    return -1 if ended_state.is_first_player() else 1
  return 0

In [ ]:
def write_data(history):
  now = datetime.now()
  os.makedirs('./data/',exist_ok=True)
  path = './data/{:04}{:02}{:02}{:02}{:02}{:02}.history'.format(now.year,now.month,now.day,now.hour,now.minute,now.second)
  with open(path,mode='wb') as f:
    pickle.dump(history,f)

In [ ]:
def play(model):
  history = []
  state = State()
  while True:
    if state.is_done():
      break
    
    scores = pv_mcts_scores(model,state,SP_TEMPERATURE)

    policies = [0]*DN_OUTPUT_SIZE
    for action,policy in zip(state.legal_actions(),scores):
      policies[action] = policy
    history.append([[state.pieces,state.enemy_pieces],policies,None])

    action = np.random.choice(state.legal_actions(),p=scores)

    state = state.next(action)

  value = first_player_value(state)
  for i in range(len(history)):
    history[i][2] = value
    value = -value
  return history

In [ ]:
def self_play():
  history = []
  model = load_model('./model/best.h5')

  for i in range(SP_GAME_COUNT):

    h = play(model)
    history.extend(h)

    print('\rSelfPlay {}/{}'.format(i+1,SP_GAME_COUNT),end='')
  print('')

  write_data(history)

  K.clear_session()
  del model

In [ ]:
if __name__ == '__main__':
  self_play()

OSError: ignored